# AlphaFold models for new DisProt set

We also want to investigate how the spearman correlation between pLDDT and true labels behaves for all sequences where we have true labels. This can then be compared with the pLDDT-IUPred correlation.

In [2]:
# We downloaded the files in the different_disorder_datasets notebook